this is a **test** cell. you can add a [link](https://docs.python.org/3/tutorial/classes.html#a-first-look-at-classes) there. sometimes, you would like to use a *italics* format. i am not sure what `inline really` means.
>Also, you can blockquote somethiend.ffffffffffffffffffff aaaaaaaaaaaaaaaa aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa dkdfjdfjd

**install notebook extensions: pip install jupyter_contrib_nbextensions && jupyter contrib nbextension install --user**

list of laTex mathematical symbols
https://oeis.org/wiki/List_of_LaTeX_mathematical_symbols

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from scipy.stats import binom
# import  statsmodels.stats.multitest as multitest
import itertools


# plotly.offline.init_notebook_mode(connected=True)
# Standard plotly imports
# import chartstudio.plotly as py
# import plotly.plotly as py
# import pandas as pd
# import plotly
# import plotly.graph_objs as go
# from plotly.offline import iplot

# from tqdm import tnrange, tqdm_notebook
# plotly.io.orca.config.port = 8999

# from plotly import tools
# cufflinks binds plotly directly to python dataframe
# import cufflinks as cf
# cf.go_offline()
# # Set the global theme
# cf.set_config_file(world_readable=True, theme='pearl', offline=True)
# plotly.offline.init_notebook_mode(connected=True)


%matplotlib inline
%reload_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# understand groupby object

In [43]:
def process_grp(grp):
    _df_or_sr = grp.apply(lambda col: 
                          ';'.join(col.unique().tolist()) 
                          if len(col.unique())==1 
                          else ';'.join(col.tolist()))
    _df = _df_or_sr.to_frame().T
    assert _df.shape[0] == 1
    return _df

dft = pd.DataFrame([['a', 'gg', 'dd'], 
                    ['a', 'gg', 'ee'],
                    ['b', 'hh', 'ee']], 
                   columns=['x', 'y', 'z'])
dft = dft.astype(str); dft

# groupby object is a list of tuples, first element is the groupby col unique values
l = list(dft.groupby('x'))
t0 = l[0][0]; t0
t1 = l[0][1]; t1; type(t1) #df
t1 = l[1][1]; t1; type(t1) #series

# apply default to columns axis=0
t1.apply(lambda col:';'.join(col.unique().tolist()) 
         if len(col.unique())==1 
         else ';'.join(col.tolist()), axis=1).to_frame().T

# apply on groupby object is really apply on each data frame resulted from the groupby operation
l[0][1].apply(lambda col: 
              ';'.join(col.unique().tolist()) 
              if len(col.unique())==1 
              else ';'.join(col.tolist()),
              axis=0).to_frame().T

dff = dft.groupby('x')[['y', 'z']].apply(process_grp)
dff
dff = dff.reset_index(level=1, drop=True)
dff


,x,y,z
0,a,gg,dd
1,a,gg,ee
2,b,hh,ee


'a'

,x,y,z
0,a,gg,dd
1,a,gg,ee


pandas.core.frame.DataFrame

,x,y,z
2,b,hh,ee


pandas.core.frame.DataFrame

,2
0,b;hh;ee


,x,y,z
0,a,gg,dd;ee


,,y,z
x,,,
a,0,gg,dd;ee
b,0,hh,ee


,y,z
x,,
a,gg,dd;ee
b,hh,ee


### subset dataframe based on values

In [9]:
df = pd.DataFrame({'A':[1, 0, 2, -2, 1, 0], 
                   'B':[1, 0, 0, 2, 1, 0],
                   'C':[1, 0, 1, 0, 1, 0],
                   'D':[1, 0, 0, 0, 1, 0], 
                   'E':[1, 0, 2, 0, 1, 0], 
                   'F':[1, 0, -2, 2, 1, 0]})
df
# find index and columns for values= 2 or -2
# use .loc, isin, any, and specify axis
vals = [2, -2]
df.loc[df.isin(vals).any(axis=1), df.isin(vals).any()]

,A,B,C,D,E,F
0,1,1,1,1,1,1
1,0,0,0,0,0,0
2,2,0,1,0,2,-2
3,-2,2,0,0,0,2
4,1,1,1,1,1,1
5,0,0,0,0,0,0


,A,B,E,F
2,2,0,2,-2
3,-2,2,0,2


# explode and expand dataframe

In [32]:
dft = pd.DataFrame(np.arange(8).reshape(2,4))
dft.head(2)
dft[4] = [[[100, 200, 300], [44,55, 66]], [[5,6, 7]]]
dft
# explode list in a column into multiple rows
dft = dft.explode(4)
dft

,0,1,2,3
0,0,1,2,3
1,4,5,6,7


,0,1,2,3,4
0,0,1,2,3,"[[100, 200, 300], [44, 55, 66]]"
1,4,5,6,7,"[[5, 6, 7]]"


,0,1,2,3,4
0,0,1,2,3,"[100, 200, 300]"
0,0,1,2,3,"[44, 55, 66]"
1,4,5,6,7,"[5, 6, 7]"


In [47]:
pd.DataFrame(dft[4].values.tolist(), index=pd.Int64Index([3,3,6]))
pd.DataFrame(dft[4].values.tolist(), index=[3,3,6])

,0,1,2
3,100,200,300
3,44,55,66
6,5,6,7


,0,1,2
3,100,200,300
3,44,55,66
6,5,6,7


In [48]:
# explode list in a column into multiple rows
dfm = dft.explode(4)
dfm

# expand list in a column into multiple columns
dft[[5,6,7]] = pd.DataFrame(dft[4].values.tolist(), index=dft.index)
dft

,0,1,2,3,4
0,0,1,2,3,100
0,0,1,2,3,200
0,0,1,2,3,300
0,0,1,2,3,44
0,0,1,2,3,55
0,0,1,2,3,66
0,0,1,2,3,100
0,0,1,2,3,200
0,0,1,2,3,300
0,0,1,2,3,44


,0,1,2,3,4,5,6,7
0,0,1,2,3,"[100, 200, 300]",100,200,300
0,0,1,2,3,"[44, 55, 66]",44,55,66
1,4,5,6,7,"[5, 6, 7]",5,6,7


In [22]:
pd.DataFrame(dft[4].values.tolist(), index=dft.index)

,0,1,2
0,100,200,300
0,44,55,66
1,5,6,7


### read_csv, columns, shape, and head
read demo file into a pandas dataframe, show column headers, number of rows and columns of the dataframe, show first 2 rows

In [2]:
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/notebook/demo.csv'
df = pd.read_csv(f)
df.columns
df.shape
df.head(2)

Index(['chr', 'start', 'end', 'patients', 'num_strelka_patients', 'gene',
       'feature_of_interest', 'num_normal_patients', 'num_mutationseq_patient',
       'Reference_Allele', 'Tumor_Seq_Allele2', 'cum_pos', 'feature_colors',
       'Clade', 'A7_count', 'A9_count'],
      dtype='object')

(174, 16)

,chr,start,end,patients,num_strelka_patients,gene,feature_of_interest,num_normal_patients,num_mutationseq_patient,Reference_Allele,Tumor_Seq_Allele2,cum_pos,feature_colors,Clade,A7_count,A9_count
0,3,183675458,183675459,S_99020;S_99120;S_99195,3,ABCC5,active_enhancer,0,5,C,T,676125452,green,A7;A9;A7,2,1
1,18,19268412,19268413,S_99120;S_99170;S_99197;S_99326;S_99354;S_9939...,8,ABHD3,active_enhancer,0,8,T,A,2600635486,green,A9;A7;A9;A7;A7;A9;A9;A9,3,5


### split a column and get first element of each row

In [51]:
# need to use str method, you can not split on a pandas series
# you can specify n: number of max splits, and expand=T/F to expand splits into new columns, df or series
# you need to keep using string method to series into a stringmethod and then split or join
s = pd.Series(['BCADDAEEAKK', 'OOAPPAATTY'])
s, type(s)
so = s.str.split('A', n=2).str[:2].str.join('-')
so, type(so)
# df['gene_new'] = df.gene.str.split('A', n=2).str[:3].str.join('-')


(0    BCADDAEEAKK
 1     OOAPPAATTY
 dtype: object, pandas.core.series.Series)

(0    BC-DD
 1    OO-PP
 dtype: object, pandas.core.series.Series)

### np.where
annotate rows based on the values in relationship between two columns. For example, if A7_count > A9_count, annotate the entry as red; otherwise assign a color of blue.

In [70]:
df['clade_colors'] = np.where((df.A7_count>df.A9_count), 'red', 'blue')
df.head(2)

,chr,start,end,patients,num_strelka_patients,gene,feature_of_interest,num_normal_patients,num_mutationseq_patient,Reference_Allele,Tumor_Seq_Allele2,cum_pos,feature_colors,Clade,A7_count,A9_count,clade_colors
0,3,183675458,183675459,S_99020;S_99120;S_99195,3,ABCC5,active_enhancer,0,5,C,T,676125452,green,A7;A9;A7,2,1,red
1,18,19268412,19268413,S_99120;S_99170;S_99197;S_99326;S_99354;S_9939...,8,ABHD3,active_enhancer,0,8,T,A,2600635486,green,A9;A7;A9;A7;A7;A9;A9;A9,3,5,blue


### pd.crosstab
crosstab can calculate frequency

1) we can count the frequency of all chromosome and hotspot type combinations

2) we can also calculate the relative frequencies across rows, columns, or all.

In [34]:
pd.crosstab??

In [12]:
pd.crosstab(index=df['chr'], columns=df['feature_of_interest']).head(2)
pd.crosstab(index=df['chr'], columns=df['feature_of_interest'], normalize='columns').head(2)
pd.crosstab(index=df['chr'], columns=df['feature_of_interest'], normalize='all').head(2)
df.pivot_table(index='chr', columns='feature_of_interest', aggfunc='size').head(2)

feature_of_interest,active_enhancer,other,potential_promoter
chr,,,
1,2,12,0
2,0,13,1


feature_of_interest,active_enhancer,other,potential_promoter
chr,,,
1,0.083333,0.086957,0.000000
2,0.000000,0.094203,0.083333


feature_of_interest,active_enhancer,other,potential_promoter
chr,,,
1,0.011494,0.068966,0.000000
2,0.000000,0.074713,0.005747


feature_of_interest,active_enhancer,other,potential_promoter
chr,,,
1,2.0,12.0,NaN
2,NaN,13.0,1.0


### merge two dataframe based on multiple columns

In [71]:
dff = pd.merge(df, df3, left_on=['gene', 'chr', 'start_x', 'Reference_Allele', 'Tumor_Seq_Allele2'], 
               right_on=['gene', 'chr', 'start_x', 'Reference_Allele', 'Tumor_Seq_Allele2'], how='left')


NameError: name 'df3' is not defined

### modify values in a column, use map or regex
 map a column to a new column with different values based on a predifined dictionary 
 
 regex can be very powerful: https://docs.python.org/2/library/re.html

In [ ]:
df['indel_colors'] = df.KRAS_snvs_indels.map({'N':'grey', 'Y':'red', 'germline':'black'})
dfn['patients_new'] = list(map(lambda x: x.replace('_T', ''), s))


In [ ]:
df_cov.loc['clinical_stage',:] = df_cov.loc['clinical_stage',:].str.replace('[AB]\d*', '')

In [ ]:
# wide to long transformation, break entries in a column into multiple rows 
# string split to expand a dataframe to long form by splitting values in a column
s = df.patients.str.split(';', expand=True).stack()
idx = s.index.get_level_values(0)
dfn = df.loc[idx]
dfn['new_col'] = list(s)


In [ ]:
df['cumsum_num_strelka'] = df.sort_values(['chr', 'start_x']).num_strela_patients.cumsum()
# cumulative sum is useful, for example calculate the cumulative genome positions when plotting events along the genome

In [ ]:
p = np.argmax(p_matrix, axis=1)
# get the index of max value in a ndarray

### groupby and aggregate
groupby_aggregate is a powerful way to manipulate data. For instance, to get the mean, minimum, maxmum, and standard deviation of the aggregated series. The syntax is df.groupby(['group col1', 'group col2].agg({'aggregate col1':[min, max, std], 'aggregate col2':[agg_func1, agg_func2]}). This gives you a both row and column multiindexed dataframe. you could rename the columns and reset the index to flatten the index so that following manipulation is easier.

In [ ]:
df3 = df2.astype(str).groupby(['gene', 'chr', 'start_x', 'Reference_Allele', 'Tumor_Seq_Allele2']).\
agg({'hiv_status':[';'.join, count_pos, count_neg], 'previous_final_hist':';'.join, \
     'Clade':[';'.join, count_A7, count_A9]}).reset_index()
df3.head(3)


In [ ]:
df2['barcode'] = df2[['Sentrix Barcode', 'Sample Section']].astype(str).apply(lambda x: '_'.join(x), axis=1) 
dfy['region'] = dfy[['Chrom', 'left', 'right']].astype(str).apply(lambda x: x[0]+':'+x[1]+'-'+x[2], axis=1)
# join two columns

In [ ]:
barcode_dict = pd.Series(df2.sample_id.values,index=df2.barcode).to_dict()
# fastest way to convert two df columns to a dictionary

In [ ]:
dfy = dfy[~dfy.pos.astype(str).str.contains('\(')]
# drop rows based on values in a column that matches a pattern
# replace only a few values in a column, you can give a list of things you would like to be replaced
df['chr'] = df['chr'].replace(['X'], 23)

this is new to me that you can take a column (series) and then groupby the other columns (other series with the same index)

In [15]:
# Create dataframe
raw_data = {'regiment': ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks', 'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons', 'Scouts', 'Scouts', 'Scouts', 'Scouts'],
        'company': ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd','1st', '1st', '2nd', '2nd'],
        'name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze', 'Jacon', 'Ryaner', 'Sone', 'Sloan', 'Piger', 'Riani', 'Ali'],
        'preTestScore': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70]}
df = pd.DataFrame(raw_data, columns = ['regiment', 'company', 'name', 'preTestScore', 'postTestScore'])
df

# Create a groupby variable that groups preTestScores by regiment
groupby_regiment = df['preTestScore'].groupby(df['company'])
groupby_regiment

# Use list() to show what a grouping looks like
list(df['preTestScore'].groupby(df['company']))

df['preTestScore'].groupby(df['company']).describe()

# Mean of each regiment's preTestScore
# Mean preTestScores grouped by regiment and company
groupby_regiment.mean()



# Mean preTestScores grouped by regiment and company without heirarchical indexing
df['preTestScore'].groupby([df['regiment'], df['company']]).mean().unstack()

# Group the entire dataframe by regiment and company
df.groupby(['regiment', 'company']).mean().unstack()

# Number of observations in each regiment and company
df.groupby(['regiment', 'company']).size().unstack()

,regiment,company,name,preTestScore,postTestScore
0,Nighthawks,1st,Miller,4,25
1,Nighthawks,1st,Jacobson,24,94
2,Nighthawks,2nd,Ali,31,57
3,Nighthawks,2nd,Milner,2,62
4,Dragoons,1st,Cooze,3,70
5,Dragoons,1st,Jacon,4,25
6,Dragoons,2nd,Ryaner,24,94
7,Dragoons,2nd,Sone,31,57
8,Scouts,1st,Sloan,2,62
9,Scouts,1st,Piger,3,70


[('1st', 0     4
  1    24
  4     3
  5     4
  8     2
  9     3
  Name: preTestScore, dtype: int64), ('2nd', 2     31
  3      2
  6     24
  7     31
  10     2
  11     3
  Name: preTestScore, dtype: int64)]

,count,mean,std,min,25%,50%,75%,max
company,,,,,,,,
1st,6.0,6.666667,8.524475,2.0,3.00,3.5,4.00,24.0
2nd,6.0,15.500000,14.652645,2.0,2.25,13.5,29.25,31.0


company
1st     6.666667
2nd    15.500000
Name: preTestScore, dtype: float64

company,1st,2nd
regiment,,
Dragoons,3.5,27.5
Nighthawks,14.0,16.5
Scouts,2.5,2.5


preTestScore       postTestScore      
company             1st   2nd           1st   2nd
regiment                                         
Dragoons            3.5  27.5          47.5  75.5
Nighthawks         14.0  16.5          59.5  59.5
Scouts              2.5   2.5          66.0  66.0

company,1st,2nd
regiment,,
Dragoons,2,2
Nighthawks,2,2
Scouts,2,2


### pivot, pivot_table, and crosstab, crosstab is a function, not a pandas method

1) pivot only reshapes dataframe, does not aggregate. can always be replaced by pivot_table

2) always use pivot_table, need to provide aggfunc even if you just want to reshape the data

In [36]:
def times100(x):
    return 100*x
df = pd.read_csv('/projects/trans_scratch/validations/workspace/szong/Cervical/notebook/animals_long.txt')
df.head(2)
df

,alphas,species,prevalence
0,[0.1 0.1 0.1],lions,0.492063
1,[1 1 1],lions,0.444444


,alphas,species,prevalence
0,[0.1 0.1 0.1],lions,0.492063
1,[1 1 1],lions,0.444444
2,[5 5 5],lions,0.380952
3,[15 15 15],lions,0.352941
4,[0.1 0.1 0.1],tigers,0.333333
5,[1 1 1],tigers,0.333333
6,[5 5 5],tigers,0.333333
7,[15 15 15],tigers,0.333333
8,[0.1 0.1 0.1],bears,0.174603
9,[1 1 1],bears,0.222222


In [40]:
df2 = df.pivot(index='alphas', columns='species', values='prevalence').reset_index()
df2

species,alphas,bears,lions,tigers
0,[0.1 0.1 0.1],0.174603,0.492063,0.333333
1,[1 1 1],0.222222,0.444444,0.333333
2,[15 15 15],0.313725,0.352941,0.333333
3,[5 5 5],0.285714,0.380952,0.333333


In [42]:
pd.melt(df2, id_vars='alphas', value_vars=['bears', 'lions'], value_name='ori_prevalence', var_name='animals')

,alphas,animals,ori_prevalence
0,[0.1 0.1 0.1],bears,0.174603
1,[1 1 1],bears,0.222222
2,[15 15 15],bears,0.313725
3,[5 5 5],bears,0.285714
4,[0.1 0.1 0.1],lions,0.492063
5,[1 1 1],lions,0.444444
6,[15 15 15],lions,0.352941
7,[5 5 5],lions,0.380952


In [38]:
# pivot a table, pivot_table?
df3 = df.pivot_table(index='alphas', columns='species', values='prevalence', aggfunc=lambda x:x)# aggfunc=times100)
df3

species,bears,lions,tigers
alphas,,,
[0.1 0.1 0.1],0.174603,0.492063,0.333333
[1 1 1],0.222222,0.444444,0.333333
[15 15 15],0.313725,0.352941,0.333333
[5 5 5],0.285714,0.380952,0.333333


In [43]:
### pivot_table for non-numerical values, long to wide transformation
df1 = pd.read_csv('/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/validation/strelka_classic_vcf_files.txt.test')
df1.head(2)

,patient,vcf,type
0,HTMCP-03-06-02025,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,indels
1,HTMCP-03-06-02025,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,snvs


In [44]:
# both works, pivot_table for non_numerical values, long to wide transformation
# df2 = pd.pivot_table(df1, index='patient', columns='type', values='vcf', aggfunc=lambda x: str(x.values[0])).reset_index()
df2 = pd.pivot_table(df1, index='patient', columns='type', values='vcf', aggfunc=lambda x: ''.join([str(v) for v in x])).reset_index()
df2.columns = ['patient', 'indels', 'snvs']
df2.head(2)

,patient,indels,snvs
0,HTMCP-03-06-02025,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
1,HTMCP-03-06-02044,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...


In [6]:

df2 = pd.pivot_table(df1, index='patient', columns='type', values='vcf', aggfunc=lambda x: ''.join([str(v) for v in x])).reset_index()

Index(['chr', 'start', 'end', 'patients', 'num_strelka_patients', 'gene',
       'feature_of_interest', 'num_normal_patients', 'num_mutationseq_patient',
       'Reference_Allele', 'Tumor_Seq_Allele2', 'cum_pos', 'feature_colors',
       'Clade', 'A7_count', 'A9_count'],
      dtype='object')

(174, 16)

,chr,start,end,patients,num_strelka_patients,gene,feature_of_interest,num_normal_patients,num_mutationseq_patient,Reference_Allele,Tumor_Seq_Allele2,cum_pos,feature_colors,Clade,A7_count,A9_count
0,3,183675458,183675459,S_99020;S_99120;S_99195,3,ABCC5,active_enhancer,0,5,C,T,676125452,green,A7;A9;A7,2,1
1,18,19268412,19268413,S_99120;S_99170;S_99197;S_99326;S_99354;S_9939...,8,ABHD3,active_enhancer,0,8,T,A,2600635486,green,A9;A7;A9;A7;A7;A9;A9;A9,3,5


# apply, applymap, map

Summing up, apply works on a row / column basis of a DataFrame, applymap works element-wise on a DataFrame, and map works element-wise on a Series.

### melt and stack
1) melt and stack reshape data in exactly the same way.

2) melt: two parameters, id_vars which are the column names that are to remain vertical (and not reshaped) and value_vars which are the column names to be reshaped into a single column.

3) use melt over stack because melt allows you to rename columns and avoids multiindex.


In [2]:
# melt wide form to long form, it is like unpivot. pivot long to wide
value_df = pd.read_csv('/projects/trans_scratch/validations/workspace/szong/Cervical/notebook/animals.txt')
print(value_df)
melted = pd.melt(value_df, id_vars = 'alphas', value_vars=['lions', 'tigers', 'bears'], value_name='prevalence',
        var_name = 'species')
print(melted)

      lions    tigers     bears         alphas
0  0.492063  0.333333  0.174603  [0.1 0.1 0.1]
1  0.444444  0.333333  0.222222        [1 1 1]
2  0.380952  0.333333  0.285714        [5 5 5]
3  0.352941  0.333333  0.313725     [15 15 15]
           alphas species  prevalence
0   [0.1 0.1 0.1]   lions    0.492063
1         [1 1 1]   lions    0.444444
2         [5 5 5]   lions    0.380952
3      [15 15 15]   lions    0.352941
4   [0.1 0.1 0.1]  tigers    0.333333
5         [1 1 1]  tigers    0.333333
6         [5 5 5]  tigers    0.333333
7      [15 15 15]  tigers    0.333333
8   [0.1 0.1 0.1]   bears    0.174603
9         [1 1 1]   bears    0.222222
10        [5 5 5]   bears    0.285714
11     [15 15 15]   bears    0.313725


In [3]:
# use stack can achieve identical results, stack works with index
value_df.set_index('alphas').stack()

alphas               
[0.1 0.1 0.1]  lions     0.492063
               tigers    0.333333
               bears     0.174603
[1 1 1]        lions     0.444444
               tigers    0.333333
               bears     0.222222
[5 5 5]        lions     0.380952
               tigers    0.333333
               bears     0.285714
[15 15 15]     lions     0.352941
               tigers    0.333333
               bears     0.313725
dtype: float64

minimal sufficient python apis: will go through them at some point
Attributes
columns
dtypes
index
shape
T
values

Aggregation Methods
all
any
count
describe
idxmax
idxmin
max
mean
median
min
mode
nunique
sum
std
var

Non-Aggretaion Statistical Methods
abs
clip
corr
cov
cummax
cummin
cumprod
cumsum
diff
nlargest
nsmallest
pct_change
prod
quantile
rank
round

Subset Selection
head
iloc
loc
tail

Missing Value Handling
dropna
fillna
interpolate
isna
notna

Grouping
expanding
groupby
pivot_table
resample
rolling

Joining Data
append
merge

Other
asfreq
astype
copy
drop
drop_duplicates
equals
isin
melt
plot
rename
replace
reset_index
sample
select_dtypes
shift
sort_index
sort_values
to_csv
to_json
to_sql
Functions
pd.concat
pd.crosstab
pd.cut
pd.qcut
pd.read_csv
pd.read_json
pd.read_sql
pd.to_datetime
pd.to_timedelta

### removed duplicate index, this works for multiple index as well

In [61]:
# use gistic refgenefile including both the cytoband and ref genes
f1 = '/projects/da_workspace/software/gistic/rg.txt'
df1 = pd.read_csv(f1)
df1.head(2)
# df1 = df1.set_index(['chrn', 'start', 'nd'])
df1.shape

,refseq,gene,symb,locus_id,chr,strand,start,nd,cds_start,cds_end,status,chrn
0,NM_000014,alpha-2-macroglobulin precursor,A2M,2,chr12,0,9220303,9268558,9220418,9268445,Reviewed,12
1,NM_000015,arylamine N-acetyltransferase 2,NAT2,10,chr8,1,18248754,18258723,18257513,18258386,Reviewed,8


(40430, 12)

In [62]:
s = pd.Series(df1.head(700).index.duplicated())
s[s].index

Int64Index([], dtype='int64')

In [67]:
df1.iloc[607:615]
df1[df1.nd == 169680843]

,refseq,gene,symb,locus_id,chr,strand,start,nd,cds_start,cds_end,status,chrn
607,NM_000643,glycogen debranching enzyme isoform 1,AGL,178,chr1,1,100316044,100389579,100316598,100387207,Reviewed,1
608,NM_000644,glycogen debranching enzyme isoform 1,AGL,178,chr1,1,100316044,100389579,100316598,100387207,Reviewed,1
609,NM_000645,glycogen debranching enzyme isoform 2,AGL,178,chr1,1,100326765,100389579,100327027,100387207,Reviewed,1
610,NM_000646,glycogen debranching enzyme isoform 3,AGL,178,chr1,1,100316530,100389579,100318225,100387207,Reviewed,1
611,NM_000651,complement receptor type 1 isoform S precursor,CR1,1378,chr1,1,207669472,207815110,207669612,207812754,Reviewed,1
612,NM_000655,L-selectin precursor,SELL,6402,chr1,0,169659805,169680843,169660924,169680678,Reviewed,1
613,NM_000657,apoptosis regulator Bcl-2 beta isoform,BCL2,596,chr18,0,60985186,60986613,60985281,60985899,Reviewed,18
614,NM_000658,autoimmune regulator isoform 2,AIRE,326,chr21,1,45710273,45718110,45710689,45717610,Reviewed,21


,refseq,gene,symb,locus_id,chr,strand,start,nd,cds_start,cds_end,status,chrn
612,NM_000655,L-selectin precursor,SELL,6402,chr1,0,169659805,169680843,169660924,169680678,Reviewed,1


In [41]:
df1.loc[~df1.index.duplicated(keep='first')].shape

,refseq,gene,symb,locus_id,chr,strand,start,nd,cds_start,cds_end,status,chrn
0,NM_000014,alpha-2-macroglobulin precursor,A2M,2,chr12,0,9220303,9268558,9220418,9268445,Reviewed,12
1,NM_000015,arylamine N-acetyltransferase 2,NAT2,10,chr8,1,18248754,18258723,18257513,18258386,Reviewed,8


(40430, 9)

(29556, 9)

### use pd.cut to bin continuous values

In [40]:
# binning a column with continuous values into a categorical value
import seaborn as sns
import time
import pandas as pd

data = sns.load_dataset('iris')

class_list = pd.cut(x=data.petal_length,
                   bins=[0, 2, 5, 100],
                   include_lowest=True,
                   labels=[1, 2, 3]).astype(int)
class_list[:3]

0    1
1    1
2    1
Name: petal_length, dtype: int64

In [1]:
import pandas as pd
import collections

In [2]:
df = pd.read_excel("https://github.com/chris1610/pbpython/blob/master/data/sample-salesv3.xlsx?raw=True")
df["date"] = pd.to_datetime(df['date'])
df.head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2014-01-01 10:00:47
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2014-01-01 15:05:22
4,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26,2014-01-01 23:26:55


In [3]:
df.to_csv('example_data.txt', index=False)

In [4]:
f = 'example_data.txt'
df = pd.read_csv(f)
df.head(2)
df["date"] = pd.to_datetime(df['date'])
df.dtypes
df.head(2)

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2014-01-01 10:00:47


In [5]:
df.set_index('date').resample('Q')["ext price"].sum() # Q quarter, M month etc

date
2014-03-31    535494.66
2014-06-30    514081.99
2014-09-30    513883.87
2014-12-31    455323.80
Freq: Q-DEC, Name: ext price, dtype: float64

In [6]:
df.set_index('date').groupby('name')["ext price"].resample("M").sum().head(2)

name        date      
Barton LLC  2014-01-31     6177.57
            2014-02-28    12218.03
Name: ext price, dtype: float64

In [7]:
df.groupby(['name', pd.Grouper(key='date', freq='M')])['ext price'].sum().head(2)

name        date      
Barton LLC  2014-01-31     6177.57
            2014-02-28    12218.03
Name: ext price, dtype: float64

In [8]:
df.groupby(['name', pd.Grouper(key='date', freq='A-DEC')])['ext price'].sum().head(2)

name                             date      
Barton LLC                       2014-12-31    109438.50
Cronin, Oberbrunner and Spencer  2014-12-31     89734.55
Name: ext price, dtype: float64

In [9]:
df.groupby(['name', pd.Grouper(key='date', freq='W-MON')])['ext price'].sum().head(2)

name        date      
Barton LLC  2014-01-06    3380.91
            2014-01-20     224.86
Name: ext price, dtype: float64

In [10]:
df.head(2)

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2014-01-01 10:00:47


In [11]:
df[["ext price", "quantity", "unit price"]].agg(['sum', 'mean'])

,ext price,quantity,unit price
sum,2.018784e+06,36463.000000,82511.290000
mean,1.345856e+03,24.308667,55.007527


In [12]:
df.agg({'ext price': ['sum', 'mean'], 'quantity': ['sum', 'mean'], 'unit price': ['mean']})

,ext price,quantity,unit price
mean,1.345856e+03,24.308667,55.007527
sum,2.018784e+06,36463.000000,NaN


In [13]:
get_max = lambda x: x.value_counts(dropna=False).index[0]

In [14]:
df['sku'].value_counts(dropna=False).index[0]

'S2-77896'

In [15]:
df.agg({'ext price': ['sum', 'mean'], 'quantity': ['sum', 'mean'], 'unit price': ['mean'], 'sku': [get_max]})

,ext price,quantity,unit price,sku
<lambda>,NaN,NaN,NaN,S2-77896
mean,1.345856e+03,24.308667,55.007527,NaN
sum,2.018784e+06,36463.000000,NaN,NaN


In [16]:
get_max.__name__ = "most frequent"

In [17]:
df.agg({'ext price': ['sum', 'mean'], 'quantity': ['sum', 'mean'], 'unit price': ['mean'], 'sku': [get_max]})

,ext price,quantity,unit price,sku
mean,1.345856e+03,24.308667,55.007527,NaN
most frequent,NaN,NaN,NaN,S2-77896
sum,2.018784e+06,36463.000000,NaN,NaN


In [18]:
f = collections.OrderedDict([('quantity', ['sum', 'mean']), ('ext price', ['sum', 'mean']),  ('sku', [get_max])])

In [19]:
df.agg(f)

,quantity,ext price,sku
mean,24.308667,1.345856e+03,NaN
most frequent,NaN,NaN,S2-77896
sum,36463.000000,2.018784e+06,NaN


In [20]:
df1 = df.groupby(['name', 'account number']).agg({'sku':','.join}).unstack()
df1.columns = df1.columns.droplevel(0)
df1.head(2)

account number,141962,146832,163416,218895,239344,257198,307599,383080,412290,424914,527099,604255,642753,672390,688981,714466,729833,737550,740150,786968
name,,,,,,,,,,,,,,,,,,,,
Barton LLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"B1-20000,B1-50809,B1-53102,S2-16558,B1-86481,S...",NaN
"Cronin, Oberbrunner and Spencer",NaN,NaN,NaN,NaN,NaN,"B1-05914,S2-16558,S1-30248,S1-82801,B1-38851,B...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df.agg({'sku': ','.join}).head(2)

,sku
0,"B,1,-,2,0,0,0,0"
1,"S,2,-,7,7,8,9,6"


In [22]:
df = pd.DataFrame({'name':['Tom','Tom','Tom','Bill','Smith'],
                   'school':['capitol','capitol','sperling','inman','uhill'],
                   'age':[7,21,8,9,10],
                   'height':[1,11,3,5,7],
                   'weight':[5,31,3,6,8],
                   'hobby':['football', 'soccer', 'badminton', 'dance', 'tumbling']})

index_col = ['name', 'school']
print (df)
df_out = df.groupby(index_col)\
           .agg({'age':'mean', 'height':sum, 'weight':max, 'hobby':','.join})#[['age','height','weight']]
print (df_out)

    name    school  age  height  weight      hobby
0    Tom   capitol    7       1       5   football
1    Tom   capitol   21      11      31     soccer
2    Tom  sperling    8       3       3  badminton
3   Bill     inman    9       5       6      dance
4  Smith     uhill   10       7       8   tumbling
                age  height  weight            hobby
name  school                                        
Bill  inman       9       5       6            dance
Smith uhill      10       7       8         tumbling
Tom   capitol    14      12      31  football,soccer
      sperling    8       3       3        badminton


# subset dataframe

In [ ]:
df = pd.DataFrame(np.random.rand(4,7), columns = ['foo', 'bar', 'quz', 'ant', 'cat', 'sat', 'dat'], index=['v', 'w', 'y', 'z'])
df

df.loc['w','bar']

# Let's assume we have a DataFrame with the following columns:
# foo, bar, quz, ant, cat, sat, dat.
# selects all rows and all columns beginning at 'foo' up to and including 'ant'
df.loc[:, 'foo':'sat']
# foo bar quz ant cat sat

# .loc accepts the same slice notation that Python lists do for both row and columns. Slice notation being start:stop:step
# slice from 'foo' to 'cat' by every 2nd column
df.loc[:, 'foo':'cat':2]
# foo quz cat

# slice from the beginning to 'bar'
df.loc[:, :'bar']
# foo bar

# slice from 'quz' to the end by 3
df.loc[:, 'quz'::3]
# quz sat

# attempt from 'sat' to 'bar'
df.loc[:, 'sat':'bar']
# no columns returned

# slice from 'sat' to 'bar'
df.loc[:, 'sat':'bar':-1]
# sat cat ant quz bar

# slice notation is syntatic sugar for the slice function
# slice from 'quz' to the end by 2 with slice function
df.loc[:, slice('quz',None, 2)]
# quz cat dat

# select specific columns with a list
# select columns foo, bar and dat
df.loc[:, ['foo','bar','dat']]
# foo bar dat

# You can slice by rows and columns. For instance if you have 5 rows with labels v, w, x, y, z
# slice from 'w' to 'y' and 'foo' to 'ant' by 3
df.loc['w':'y', 'foo':'ant':3]
#    foo ant
# w
# x
# y

### align
align df1 to df2 based on axis(row, column or both) and join (inner, outer, left, right). this does not combine the two dataframe. Instead, it aligns the two df separately so that the row and/or column configuration of the two dataframes are the same.

https://stackoverflow.com/questions/51645195/pandas-align-function-illustrative-example

In [52]:
df1 = pd.DataFrame([[1,2,3,4], [6,7,8,9]], columns=['D', 'B', 'E', 'A'], index=[1,2])
df2 = pd.DataFrame([[10,20,30,40], [60,70,80,90], [600,700,800,900]], columns=['A', 'B', 'C', 'D'], index=[2,3,4])
df1
df2

,D,B,E,A
1,1,2,3,4
2,6,7,8,9


,A,B,C,D
2,10,20,30,40
3,60,70,80,90
4,600,700,800,900


In [53]:
a1, a2 = df1.align(df2, join='outer', axis=1)
print(a1)
print(a2)

   A  B   C  D  E
1  4  2 NaN  1  3
2  9  7 NaN  6  8
     A    B    C    D   E
2   10   20   30   40 NaN
3   60   70   80   90 NaN
4  600  700  800  900 NaN


In [54]:
a1, a2 = df1.align(df2, join='right', axis=None)
print(a1)
print(a2)

     A    B   C    D
2  9.0  7.0 NaN  6.0
3  NaN  NaN NaN  NaN
4  NaN  NaN NaN  NaN
     A    B    C    D
2   10   20   30   40
3   60   70   80   90
4  600  700  800  900


In [55]:
a1, a2 = df1.align(df2, join='inner', axis=1)
print(a1)
print(a2)

   D  B  A
1  1  2  4
2  6  7  9
     D    B    A
2   40   20   10
3   90   70   60
4  900  700  600


In [36]:
# select columns based on values in a row, use series selection or transpose the df first
mutation_f = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients_old_vcfs/smgs_reviewed_details_with_frequency.txt.20190104'
df = pd.read_csv(mutation_f, index_col='gene')
df.head(2)
s = df.loc['Putative_histology',:] == 'Adenosquamous'
s[:3]
s[s].index
df[s[s].index].head()

,hiv_pos_mfreq,hiv_neg_mfreq,mfreq,HTMCP-03-06-02260,HTMCP-03-06-02428,HTMCP-03-06-02139,HTMCP-03-06-02240,HTMCP-03-06-02262,HTMCP-03-06-02001,HTMCP-03-06-02411,...,HTMCP-03-06-02326,HTMCP-03-06-02185,HTMCP-03-06-02222,HTMCP-03-06-02197,HTMCP-03-06-02194,HTMCP-03-06-02332,HTMCP-03-06-02156,HTMCP-03-06-02046,HTMCP-03-06-02008,HTMCP-03-06-02037
gene,,,,,,,,,,,,,,,,,,,,,
HIV_status,100,100,102,Positive,Positive,Positive,Positive,Positive,Positive,Positive,...,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative
Putative_histology,100,100,102,Squamous,Squamous,Adeno,Squamous,Squamous,Squamous,Squamous,...,Adeno,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous


hiv_pos_mfreq    False
hiv_neg_mfreq    False
mfreq            False
Name: Putative_histology, dtype: bool

Index(['HTMCP-03-06-02110', 'HTMCP-03-06-02216', 'HTMCP-03-06-02203',
       'HTMCP-03-06-02176'],
      dtype='object')

,HTMCP-03-06-02110,HTMCP-03-06-02216,HTMCP-03-06-02203,HTMCP-03-06-02176
gene,,,,
HIV_status,Positive,Positive,Negative,Negative
Putative_histology,Adenosquamous,Adenosquamous,Adenosquamous,Adenosquamous
PIK3CA(35%),NaN,NaN,Non-synonymous,NaN
FAT1(19%),NaN,NaN,NaN,NaN
MLL2(15%),NaN,NaN,NaN,NaN


# groupby filter

In [14]:
dff = pd.DataFrame({'A': np.arange(8), 'B': list('aabbbbcc')})
dff

dff.groupby('B').filter(lambda x: len(x) > 2)

dff.groupby('B').filter(lambda x: len(x) > 2, dropna=False)

dff['C'] = np.arange(8)
dff

dff.groupby('B').filter(lambda x: len(x['C']) > 2)

,A,B
0,0,a
1,1,a
2,2,b
3,3,b
4,4,b
5,5,b
6,6,c
7,7,c


,A,B
2,2,b
3,3,b
4,4,b
5,5,b


,A,B
0,NaN,NaN
1,NaN,NaN
2,2.0,b
3,3.0,b
4,4.0,b
5,5.0,b
6,NaN,NaN
7,NaN,NaN


,A,B,C
0,0,a,0
1,1,a,1
2,2,b,2
3,3,b,3
4,4,b,4
5,5,b,5
6,6,c,6
7,7,c,7


,A,B,C
2,2,b,2
3,3,b,3
4,4,b,4
5,5,b,5
